# 하이버 인기브랜드 크롤러 개발하기
---
### 1. 필수 라이브러리 설치하기
#### (1) request 라이브러리 설치
- Anaconda Cloud의 작은 브라우저 역할을 하게됨
- 특정 웹사이트에서 html에서 소스코드를 그대로 가져와서 반환해주는 역할을 하게됨
- `conda install -c anaconda requests`

#### (2) BeautifulSoup 라이브러리 설치
- html 페이지에서 원하는 형태를 찾는 것에 도움이 됨
- `conda install -c anaconda beautifulsoup4`

#### (3) tqdm 라이브러리 설치
- 작업에 대한 상태를 확인할 때 유용
- `conda install -c conda-forge tqdm`

#### 윈도우에서 conda 라이브러리를 통해 위 항목을 설치하도록 한다.
- 아나콘다 prompt를 열어주는데, 설치할 때 관리자권한으로 설치하는 것을 권장
- Anaconda Prompt를 관리자 권한으로 실행해야함
- 새로운 가상환경을 만들지 않았다면 (base)라는 문구가 터미널에서 보여야 정상

### 2. 수집할 사이트 확인하기
---
- 사이트 주소: [하이버 > 랭킹 > 브랜드 > 월간](https://www.hiver.co.kr/ranking?id=296&order=monthly)

![hiver_best](https://ifh.cc/g/KR8tAK.jpg)

In [1]:
#주요 라이브러리 호출하기

import pandas as pd
from bs4 import BeautifulSoup as bs
import time
from tqdm import tqdm

In [2]:
#수집할 사이트 정보 가져오기

app_name = "하이버"
url = "https://www.hiver.co.kr/ranking?id=296&order=monthly"

#table = pd.read_html(url)
#실행결과 테이블이 없다는 것을 확인

<b>이런 경우 `Requests`를 활용하게 됨 → 이를 활용하면 이미지, CSS파일 적용없이 소스코드만을 가져오게 됨
- 우리가 수집하려는 페이지가 `get` 방식이기 때문에, 아래에서도 `requests.get`을 활용한다.
- 실행했을 때 `<Response [200]>`이라는 결과값이 나오는데, 이는 `Headers`의 `Status Code`의 200과 동일한 의미로 잘 받았다는 의미다.


이를 그냥 활용하기 어렵기 떄문에, 우리는 `beautifulsoup` 의 도움을 받게 된다.

In [4]:
import requests

response = requests.get(url)
response.status_code

200

In [5]:
from bs4 import BeautifulSoup as bs

# Warning 메시지가 뜨는 경우, lxml을 지정해주면 사라짐
# BeautifulSoup을 파싱할 때 lxml로 파싱해달라고 지정하는 옵션으로 보면됨
html = bs(response.text, "lxml")

- 확인한 결과, 보안으로 `Beautifulsoup`을 통해 소스코드에서 상품명을 가져오는 것이 불가능한 상황
- 따라서 이 경우에는 `Selenium`을 활용해서 작업할 수 있도록 한다.

#### Selenium 설치하기
- `selenium`과 `webdriver_manager`를 설치해준다.

In [7]:
import selenium
selenium.__version__

'4.8.3'

In [8]:
import webdriver_manager
webdriver_manager.__version__

'3.8.5'

In [9]:
from selenium import webdriver

driver = webdriver.Chrome('C:/Users/win10_original/Desktop/chrome_driver/chromedriver')
driver.implicitly_wait(10)
driver.get(url)

C:\Users\win10_original\AppData\Local\Temp\ipykernel_12504\128688385.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/win10_original/Desktop/chrome_driver/chromedriver')


In [10]:
from selenium.webdriver.common.by import By

html_comments = driver.find_elements(By.CSS_SELECTOR, ".css-c1cgl.e1c07x489") #이거는 실패
element = driver.find_element(By.XPATH, '//*[@id="container"]/div[2]/section[2]/div[1]/a/div/div[2]/div/strong')
element = driver.find_element(By.XPATH, '//*[@id="container"]/div[2]/section[2]/div[2]/a/div/div[2]/div/strong')


print(element.text)

블론드나인


In [14]:
# 수집한 데이터를 저장할 리스트를 생성합니다.
elements = []
ranks = []

# 초기 페이지를 설정합니다.
url = "https://www.hiver.co.kr/ranking?id=296&order=monthly"
driver.get(url)
driver.implicitly_wait(10)

# 페이지를 순차적으로 이동하며 element를 수집합니다.

for i in range(1, 101):
    rank = i
    xpath = f'//*[@id="container"]/div[2]/section[2]/div[{i}]/a/div/div[2]/div/strong'
    element = driver.find_element(By.XPATH, xpath)
    elements.append(element.text)
    ranks.append(rank)

# 데이터프레임을 생성합니다.
brand_df = pd.DataFrame({'Rank': ranks, 'Item': elements})

In [15]:
# 수집한 데이터를 저장할 리스트를 생성합니다.
elements = []
ranks = []

# 초기 페이지를 설정합니다.
url = "https://www.hiver.co.kr/ranking?id=416&order=monthly"
driver.get(url)
driver.implicitly_wait(10)

# 페이지를 순차적으로 이동하며 element를 수집합니다.

for i in range(1, 101):
    rank = i
    xpath = f'//*[@id="container"]/div[2]/section[2]/div[{i}]/a/div/div[2]/div/strong'
    element = driver.find_element(By.XPATH, xpath)
    elements.append(element.text)
    ranks.append(rank)

# 데이터프레임을 생성합니다.
shopping_df = pd.DataFrame({'Rank': ranks, 'Item': elements})

In [20]:
df_concat = pd.concat([brand_df, shopping_df], axis=0)
df_concat

,Rank,Item
0,1,블론드나인
1,2,엑스컨테이너
2,3,엑스컨테이너
3,4,앨빈클로
4,5,그랜디오시티
...,...,...
95,96,유얼마인드
96,97,에이치
97,98,베이직바이블
98,99,르위


### 수집한 데이터를 하나의 데이터 프레임으로 합치기

- `pd.concat` 기능을 활용해서 하나로 합칠 수 있음
- (주의) 기간이 긴 데이터를 수집할 때는 서버에 부담을 주지 않기 위해 time.sleep() 값을 주게 된다.

In [21]:
#파생변수 생성하기
df_concat['app'] = app_name

#브랜드명에서 공백제거하기
df_concat['brand_KR'] = df_concat['Item'].str.replace(' ','')

cols = ['app', 'Rank', 'brand_KR']
df_concat = df_concat[cols]
df_concat

,app,Rank,brand_KR
0,하이버,1,블론드나인
1,하이버,2,엑스컨테이너
2,하이버,3,엑스컨테이너
3,하이버,4,앨빈클로
4,하이버,5,그랜디오시티
...,...,...,...
95,하이버,96,유얼마인드
96,하이버,97,에이치
97,하이버,98,베이직바이블
98,하이버,99,르위


In [22]:
brand_list = pd.read_csv('brand_hosting_archive.csv')

#수집된 데이터에 LEFT JOIN
merged_df = pd.merge(df_concat, brand_list, how='left', left_on='brand_KR', right_on='brand_KR')
#merged_df = merged_df.drop(['brand_EN', 'brand_KR'], axis=1)

merged_df

,app,Rank,brand_KR,brand_EN,URL
0,하이버,1,블론드나인,BLOND9,blond9.com
1,하이버,2,엑스컨테이너,EXCONTAINER,excontainer.co.kr
2,하이버,3,엑스컨테이너,EXCONTAINER,excontainer.co.kr
3,하이버,4,앨빈클로,ALVINCLO,alvinclo.co.kr
4,하이버,5,그랜디오시티,GRANDIOSITY,list.hiphoper.combrand/GRANDIOSITY
...,...,...,...,...,...
215,하이버,96,유얼마인드,NaN,NaN
216,하이버,97,에이치,NaN,NaN
217,하이버,98,베이직바이블,NaN,NaN
218,하이버,99,르위,NaN,NaN


In [26]:
#개별몰 주소가 확인되지 않은 브랜드 목록

condition = merged_df['URL'].isnull()
result = merged_df.loc[condition, 'brand_KR'].unique()
result

array(['코드스탠다드', '어웨이크', '에스이에스티', '에즈카톤', '프롬에이투비', '모노포스', '독보남',
       '플레이마틴', '그레이즈', '자비노', '피피옴므', '블루트', '히코튼', '에밀리에', '르위', '마인드핏',
       '미스터제이슨', '노컴플렉스', '이룰', '상드르', '자시가', '스토리지', '큐쓰', '업노멀', '캔노멀',
       '베이직바이블', '플로우스푼', '위노드', '헤일로샵', '탑보이', '비비탄', '치트키', '제타22',
       '밸류젯', '우모르', '나뽀', '맨즈셀렉터', '척도', '주포유', '디스페로', '아이디에프', '엽옷이오',
       '톰엔래빗', '투아미고', '프롬엘', '뉴트렉션', '웨이든', '맨즈델리', '맨인스토어', '오오룩',
       '로댄티', '제로스샵', '덕션', '아일랜드모던', '준간', '초이스로', '블랑토', '유얼마인드', '에이치'],
      dtype=object)

### 데이터 전처리하기 및 EDA

- 중복 입력된 브랜드 필터링
- 사이트 주소 없는 브랜드 확인하기

In [27]:
import datetime

# 오늘 날짜 구하기
today = datetime.date.today()

# YYYY-MM-DD 형태의 문자열로 변환
today_str = today.strftime('%Y-%m-%d')

In [28]:
file_name = f"{app_name}_{today_str}.csv"
file_name

'하이버_2023-04-07.csv'

In [29]:
merged_df.to_csv(file_name, index=False)
pd.read_csv(file_name)

,app,Rank,brand_KR,brand_EN,URL
0,하이버,1,블론드나인,BLOND9,blond9.com
1,하이버,2,엑스컨테이너,EXCONTAINER,excontainer.co.kr
2,하이버,3,엑스컨테이너,EXCONTAINER,excontainer.co.kr
3,하이버,4,앨빈클로,ALVINCLO,alvinclo.co.kr
4,하이버,5,그랜디오시티,GRANDIOSITY,list.hiphoper.combrand/GRANDIOSITY
...,...,...,...,...,...
215,하이버,96,유얼마인드,NaN,NaN
216,하이버,97,에이치,NaN,NaN
217,하이버,98,베이직바이블,NaN,NaN
218,하이버,99,르위,NaN,NaN
